<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/SAM_LoRA_DoRA_AdaLoRA_VeLoRA_MoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip -q install git+https://github.com/facebookresearch/segment-anything.git
!wget wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth


  Preparing metadata (setup.py) ... done
--2025-06-28 15:43:03--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2025-06-28 15:43:03--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.37.111, 13.35.37.123, 13.35.37.90, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.37.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b_01ec64.pth’

sam_vit_b_01ec64.pt 100%[===================>] 357.67M  32.9MB/s    in 5.2s    

2025-06-28 15:43:08 (68.9 MB/s) - ‘sam_vit_b_01ec64.pth’ saved [375042383/375042383]

FINISHED --2025-06-28 15:43:08--
Total wall clock time: 5.3s
Downloaded: 1 files, 358M in 5.2s (68.9 MB/s)


#Identify Target Modules for Peft in SAM

In [9]:
from segment_anything import sam_model_registry

sam = sam_model_registry["vit_b"](
    checkpoint="sam_vit_b_01ec64.pth")
sam_encoder = sam.image_encoder

for name, module in sam_encoder.named_modules():
    print(name)



patch_embed
patch_embed.proj
blocks
blocks.0
blocks.0.norm1
blocks.0.attn
blocks.0.attn.qkv
blocks.0.attn.proj
blocks.0.norm2
blocks.0.mlp
blocks.0.mlp.lin1
blocks.0.mlp.lin2
blocks.0.mlp.act
blocks.1
blocks.1.norm1
blocks.1.attn
blocks.1.attn.qkv
blocks.1.attn.proj
blocks.1.norm2
blocks.1.mlp
blocks.1.mlp.lin1
blocks.1.mlp.lin2
blocks.1.mlp.act
blocks.2
blocks.2.norm1
blocks.2.attn
blocks.2.attn.qkv
blocks.2.attn.proj
blocks.2.norm2
blocks.2.mlp
blocks.2.mlp.lin1
blocks.2.mlp.lin2
blocks.2.mlp.act
blocks.3
blocks.3.norm1
blocks.3.attn
blocks.3.attn.qkv
blocks.3.attn.proj
blocks.3.norm2
blocks.3.mlp
blocks.3.mlp.lin1
blocks.3.mlp.lin2
blocks.3.mlp.act
blocks.4
blocks.4.norm1
blocks.4.attn
blocks.4.attn.qkv
blocks.4.attn.proj
blocks.4.norm2
blocks.4.mlp
blocks.4.mlp.lin1
blocks.4.mlp.lin2
blocks.4.mlp.act
blocks.5
blocks.5.norm1
blocks.5.attn
blocks.5.attn.qkv
blocks.5.attn.proj
blocks.5.norm2
blocks.5.mlp
blocks.5.mlp.lin1
blocks.5.mlp.lin2
blocks.5.mlp.act
blocks.6
blocks.6.norm1
blo

#Common Target Modules are:

attn.qkv

attn.proj

mlp.fc1

mlp.fc2

#Apply LoRA on SAM

In [21]:
from segment_anything import sam_model_registry
from peft import get_peft_model, LoraConfig

sam = sam_model_registry["vit_b"](
    checkpoint="sam_vit_b_01ec64.pth")
sam_encoder = sam.image_encoder

# LoRA configuration for SAM
lora_config = LoraConfig(
    r=8,
    #target_modules=["qkv", "proj"],  # Adjust to your SAM model inspection
    target_modules=["attn.qkv", "attn.proj", "mlp.fc1","mlp.fc2"], # SAM layer to apply PEFT
    lora_dropout=0.1,
    bias="none",
    task_type="FEATURE_EXTRACTION"
)

# Apply LoRA to SAM encoder
sam_encoder_lora = get_peft_model(sam_encoder, lora_config)

# Verify LoRA parameters
sam_encoder_lora.print_trainable_parameters()

trainable params: 442,368 || all params: 90,113,280 || trainable%: 0.4909


#Apply DoRA on SAM

In [20]:
dora_config = LoraConfig(
    r=8,  # Rank of adaptation
    # Target modules specific to vision transformers
    # target_modules=["qkv", "proj", "fc1", "fc2"],  # Adjust to your SAM model inspection
    target_modules=["attn.qkv", "attn.proj", "mlp.fc1","mlp.fc2"], # SAM layer to apply PEFT
    lora_dropout=0.1,
    use_dora=True,  # Enable DoRA (Weight-Decomposed Low-Rank Adaptation)
    bias="none",  # Don't adapt bias terms
    task_type="FEATURE_EXTRACTION"
)

# Apply LoRA to SAM encoder
sam_encoder_dora = get_peft_model(sam_encoder, dora_config)

# Verify LoRA parameters
sam_encoder_dora.print_trainable_parameters()

trainable params: 1,635,360 || all params: 91,306,296 || trainable%: 1.7911


#Apply AdaLoRA on SAM

In [19]:
from peft import AdaLoraConfig, TaskType

# Assume you have a fixed number of epochs and batches
num_epochs = 60
batches_per_epoch = 100 #len(dataloader)  # your DataLoader
total_steps = num_epochs * batches_per_epoch

# Configure AdaLoRA adapters
adalora_config = AdaLoraConfig(
    r=8,  # Low-rank matrix rank
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.1,  # Dropout rate
    # target_modules=["attn.qkv", "attn.proj"], # SAM layer to apply PEFT
    target_modules=["attn.qkv", "attn.proj", "mlp.fc1","mlp.fc2"], # SAM layer to apply PEFT
    beta1=0.85,  # Adaptive importance weighting factor
    beta2=0.85,  # Further control factor for importance weighting
    orth_reg_weight=0.5,  # Orthogonality regularization weight
    total_step=total_steps,  # Required!
    task_type="FEATURE_EXTRACTION"
)

# Apply AdaLoRA to SAM encoder
sam_encoder_adalora = get_peft_model(sam_encoder, adalora_config)

# Verify LoRA parameters
sam_encoder_adalora.print_trainable_parameters()

trainable params: 1,156,128 || all params: 90,827,064 || trainable%: 1.2729


#Apply VeLoRA on SAM

In [23]:
# Configure VeLoRA adapters (Approximating VeLoRA with Variable Low-Rank Updates)
velora_config = LoraConfig(
    r=16,  # Initial low-rank matrix rank
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout rate
    target_modules=["attn.qkv", "attn.proj"], # SAM layer to apply PEFT
    rank_pattern={"attn.qkv": 16, "attn.proj": 8},  # Simulating variable rank adjustment
    task_type="FEATURE_EXTRACTION"
)

# Apply VeLoRA to SAM encoder
sam_encoder_velora = get_peft_model(sam_encoder, velora_config)

# Verify LoRA parameters
sam_encoder_velora.print_trainable_parameters()

trainable params: 737,280 || all params: 90,408,192 || trainable%: 0.8155


#This will replace original hf peft lib. You must use a dedicated vir env for MoRA.
#Apply MoRA on SAM
installing peft MoRA

In [25]:
!git clone https://github.com/kongds/MoRA.git
%cd MoRA
!pip -q install -e ./peft-mora

Cloning into 'MoRA'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 176 (delta 57), reused 136 (delta 33), pack-reused 0 (from 0)
Receiving objects: 100% (176/176), 257.55 KiB | 16.10 MiB/s, done.
Resolving deltas: 100% (57/57), done.
/content/MoRA
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Must restart the session in colab notebook: Runtime->Restart Session

In [2]:
from peft import LoraConfig, get_peft_model
from segment_anything import sam_model_registry

sam = sam_model_registry["vit_b"](
    checkpoint="sam_vit_b_01ec64.pth")
sam_encoder = sam.image_encoder


mora_config = LoraConfig(
    r=8,
    lora_dropout=0.1,
    target_modules=["attn.qkv", "attn.proj", "mlp.fc1", "mlp.fc2"],  # linear layers only!
    task_type="FEATURE_EXTRACTION",
    use_mora=True,
    mora_type=6,
)

# Apply MoRA to SAM encoder
sam_encoder_mora = get_peft_model(sam_encoder, mora_config)

# Verify LoRA parameters
sam_encoder_mora.print_trainable_parameters()

trainable params: 437,232 || all params: 90,108,144 || trainable%: 0.48523028062813056
